In [1]:
# ============================================================================
# VERSION 1: Baseline LogisticRegression - paramètres par défaut
# ============================================================================
# Objectif: modèle le plus simple possible, sans aucune gestion du déséquilibre
#           ni ajustement de seuil
# Validation: StratifiedKFold (5 folds) pour conserver la proportion de classes
# Modèle: LogisticRegression() avec max_iter=1000, random_state=42
# Features: X_train, y_train, X_test, y_test (seront scalés avec StandardScaler)
# Seuil fixe: 0.5
# Métriques par fold: AUC-ROC, Accuracy, F1-score, Recall classe 1
# Coût métier: 10 * FN + 1 * FP (avec seuil=0.5)
# MLflow: run_name="V1_LogisticRegression_Baseline"
# Tags: version="1", model="LogisticRegression"

import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings('ignore', message='.*Failed to resolve installed pip version.*')
warnings.filterwarnings('ignore', category=FutureWarning, message='.*penalty.*deprecated.*')
warnings.filterwarnings('ignore', category=ConvergenceWarning)

# ============================================================================
# CONFIGURATION
# ============================================================================
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"
MLFLOW_EXPERIMENT_NAME = "OC_P6_Credit_Scoring"

PROJECT_VERSION = "1.0"
MODEL_NAME = "LogisticRegression"
NOTEBOOK_NAME = "04_regression"
RUN_DATE = datetime.datetime.now()

DATA_PATH = "../data/processed/"
TRAIN_FILE = "features_train.csv"
TEST_FILE = "features_test.csv"

# Configuration du modèle baseline (paramètres par défaut)
MODEL_CONFIG_V1 = {
    "max_iter": 1000,
    "random_state": 42
}

RANDOM_STATE = 42
THRESHOLD_FIXED = 0.5  # Seuil fixe pour les prédictions

print("Configuration chargée avec succès !")
print(f"MLflow Experiment: {MLFLOW_EXPERIMENT_NAME}")
print(f"Model: {MODEL_NAME}")
print(f"Fixed threshold: {THRESHOLD_FIXED}")


Configuration chargée avec succès !
MLflow Experiment: OC_P6_Credit_Scoring
Model: LogisticRegression
Fixed threshold: 0.5


In [2]:
# Configuration MLflow
from src.mlflow_config import configure_mlflow

mlflow = configure_mlflow(autolog=False)


In [3]:
# ============================================================================
# CHARGEMENT ET PRÉPARATION DES DONNÉES
# ============================================================================

# Chargement des données d'entraînement
X_train = pd.read_csv(DATA_PATH + TRAIN_FILE)
y_train = X_train.pop("TARGET")

# Chargement des données de test
X_test = pd.read_csv(DATA_PATH + TEST_FILE)
y_test = X_test.pop("TARGET")

print(f"✓ Données chargées:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"  Classe 0: {(y_train==0).sum()}, Classe 1: {(y_train==1).sum()}")

# ============================================================================
# NETTOYAGE DES DONNÉES: Suppression des colonnes avec 100% NaN
# ============================================================================
from sklearn.impute import SimpleImputer

# Vérifier les NaN
nan_train = X_train.isna().sum().sum()
nan_test = X_test.isna().sum().sum()

print(f"\n✓ Vérification initiale des NaN:")
print(f"  Total NaN in X_train: {nan_train}")
print(f"  Total NaN in X_test: {nan_test}")

# Identifier et supprimer les colonnes entièrement NaN dans X_train
empty_cols_train = X_train.columns[X_train.isna().all()].tolist()
empty_cols_test = X_test.columns[X_test.isna().all()].tolist()

print(f"\n✓ Identification des colonnes vides (100% NaN):")
print(f"  Colonnes vides dans X_train: {len(empty_cols_train)}")
print(f"  Colonnes vides dans X_test: {len(empty_cols_test)}")

# Supprimer les colonnes vides (union des deux ensembles)
cols_to_drop = set(empty_cols_train) | set(empty_cols_test)
if cols_to_drop:
    print(f"  Suppression de {len(cols_to_drop)} colonnes vides...")
    X_train = X_train.drop(columns=list(cols_to_drop))
    X_test = X_test.drop(columns=list(cols_to_drop))
    print(f"  X_train après suppression: {X_train.shape}")
    print(f"  X_test après suppression: {X_test.shape}")

# ============================================================================
# IMPUTATION DES VALEURS NaN RESTANTES AVEC LA MÉDIANE
# ============================================================================

nan_train_remaining = X_train.isna().sum().sum()
nan_test_remaining = X_test.isna().sum().sum()

print(f"\n✓ Imputation des NaN restants:")
print(f"  NaN restants in X_train: {nan_train_remaining}")
print(f"  NaN restants in X_test: {nan_test_remaining}")

if nan_train_remaining > 0 or nan_test_remaining > 0:
    print(f"  Imputation avec la médiane...")
    
    # Créer un imputer avec stratégie médiane
    imputer = SimpleImputer(strategy='median')
    
    # Fit sur X_train et transformer X_train et X_test
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)
    
    # Reconvertir en DataFrame
    X_train = pd.DataFrame(X_train_imputed, columns=X_train.columns)
    X_test = pd.DataFrame(X_test_imputed, columns=X_test.columns)
    
    print(f"  X_train après imputation: {X_train.shape}")
    print(f"  X_test après imputation: {X_test.shape}")
    print(f"  Vérification post-imputation:")
    print(f"    NaN in X_train: {X_train.isna().sum().sum()}")
    print(f"    NaN in X_test: {X_test.isna().sum().sum()}")
else:
    print(f"  Aucun NaN à imputer !")

print(f"\n✓ Données finales après nettoyage:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")


✓ Données chargées:
  X_train: (10000, 741)
  y_train: (10000,)
  X_test: (10000, 741)
  y_test: (10000,)
  Classe 0: 9225, Classe 1: 775

✓ Vérification initiale des NaN:
  Total NaN in X_train: 5146964
  Total NaN in X_test: 5106144

✓ Identification des colonnes vides (100% NaN):
  Colonnes vides dans X_train: 17
  Colonnes vides dans X_test: 1
  Suppression de 17 colonnes vides...
  X_train après suppression: (10000, 724)
  X_test après suppression: (10000, 724)

✓ Imputation des NaN restants:
  NaN restants in X_train: 4976964
  NaN restants in X_test: 4936311
  Imputation avec la médiane...
  X_train après imputation: (10000, 724)
  X_test après imputation: (10000, 724)
  Vérification post-imputation:
    NaN in X_train: 0
    NaN in X_test: 0

✓ Données finales après nettoyage:
  X_train: (10000, 724)
  y_train: (10000,)
  X_test: (10000, 724)
  y_test: (10000,)


In [4]:
# ============================================================================
# STANDARDISATION DES FEATURES
# ============================================================================
# La régression logistique est sensible à l'échelle des features
# Utiliser StandardScaler (fit sur train, transform sur test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reconvertir en DataFrame pour conserver les noms de colonnes
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print(f"\n✓ Features standardisées (StandardScaler):")
print(f"  Shape train: {X_train_scaled.shape}")
print(f"  Mean: {X_train_scaled.mean().mean():.8f} (≈ 0)")
print(f"  Std:  {X_train_scaled.std().mean():.6f} (≈ 1)")



✓ Features standardisées (StandardScaler):
  Shape train: (10000, 724)
  Mean: -0.00000000 (≈ 0)
  Std:  0.874353 (≈ 1)


In [5]:
# ============================================================================
# CROSS-VALIDATION: LogisticRegression V1 Baseline
# ============================================================================
# StratifiedKFold (5 folds) pour conserver la proportion de classes
# Seuil fixe = 0.5 pour les prédictions (pas d'optimisation)

from src.mlflow_config import configure_mlflow

mlflow = configure_mlflow(autolog=False)

# Terminer tout run actif avant de commencer
mlflow.end_run()

RUN_NAME_V1 = "V1_LogisticRegression_Baseline"

fold_results = []

with mlflow.start_run(run_name=RUN_NAME_V1):
    # ========== Logging des paramètres et tags ==========
    mlflow.log_params(MODEL_CONFIG_V1)
    mlflow.set_tag("version", "1")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("notebook", NOTEBOOK_NAME)
    mlflow.set_tag("phase", "baseline_cv")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("scaling", "StandardScaler")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # ========== StratifiedKFold (5 folds) ==========
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train_scaled, y_train), start=1):
        X_tr, X_val = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # ========== Entraînement ==========
        model = LogisticRegression(**MODEL_CONFIG_V1)
        model.fit(X_tr, y_tr)
        
        # ========== Prédictions ==========
        y_val_proba = model.predict_proba(X_val)[:, 1]  # Probabilités classe 1
        y_val_pred = (y_val_proba >= THRESHOLD_FIXED).astype(int)  # Seuil fixe 0.5
        
        # ========== Métriques ==========
        auc = roc_auc_score(y_val, y_val_proba)
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        
        # ========== Coût métier (seuil=0.5) ==========
        tn, fp, fn, tp = confusion_matrix(y_val, y_val_pred).ravel()
        cost = 10 * fn + 1 * fp
        
        fold_results.append({
            "fold": fold_idx,
            "auc": auc,
            "accuracy": accuracy,
            "f1_score": f1,
            "recall_class1": recall,
            "business_cost_min": cost,
            "optimal_threshold": THRESHOLD_FIXED,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn
        })
        
        print(f"Fold {fold_idx}/5 | AUC={auc:.4f} | Acc={accuracy:.4f} | "
              f"F1={f1:.4f} | Recall={recall:.4f} | Cost={cost:.0f}")
    
    # ========== Agrégation des résultats ==========
    cv_results_df = pd.DataFrame(fold_results)
    
    metrics_mean = {
        "auc": cv_results_df["auc"].mean(),
        "f1_score": cv_results_df["f1_score"].mean(),
        "recall_class1": cv_results_df["recall_class1"].mean(),
        "business_cost_min": cv_results_df["business_cost_min"].mean(),
        "optimal_threshold": THRESHOLD_FIXED,
    }
    
    metrics_std = {
        "auc": cv_results_df["auc"].std(),
        "f1_score": cv_results_df["f1_score"].std(),
        "recall_class1": cv_results_df["recall_class1"].std(),
        "business_cost_min": cv_results_df["business_cost_min"].std(),
    }
    
    # ========== Logging dans MLFlow ==========
    # Utiliser les MÊMES noms que le schéma standard MLflow (sans préfixe)
    mlflow.log_metric("auc", metrics_mean["auc"])
    mlflow.log_metric("f1_score", metrics_mean["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean["optimal_threshold"])
    
    # Log artefact JSON avec détails par fold
    mlflow.log_dict(cv_results_df.to_dict(orient="records"), "cv_results_per_fold.json")
    
    print("\n✓ Cross-Validation LogisticRegression V1 terminée")
    print(f"  AUC moyen: {metrics_mean['auc']:.4f} ± {metrics_std['auc']:.4f}")
    print(f"  F1 moyen: {metrics_mean['f1_score']:.4f} ± {metrics_std['f1_score']:.4f}")
    print(f"  Recall moyen: {metrics_mean['recall_class1']:.4f} ± {metrics_std['recall_class1']:.4f}")
    print(f"  Coût métier moyen: {metrics_mean['business_cost_min']:.2f} ± {metrics_std['business_cost_min']:.2f}")
    print(f"  Seuil optimal: {metrics_mean['optimal_threshold']:.2f}")


Fold 1/5 | AUC=0.6995 | Acc=0.9170 | F1=0.0568 | Recall=0.0323 | Cost=1516
Fold 2/5 | AUC=0.6972 | Acc=0.9225 | F1=0.1243 | Recall=0.0710 | Cost=1451
Fold 3/5 | AUC=0.7036 | Acc=0.9160 | F1=0.0562 | Recall=0.0323 | Cost=1518
Fold 4/5 | AUC=0.7061 | Acc=0.9140 | F1=0.0444 | Recall=0.0258 | Cost=1531
Fold 5/5 | AUC=0.6984 | Acc=0.9145 | F1=0.0339 | Recall=0.0194 | Cost=1539

✓ Cross-Validation LogisticRegression V1 terminée
  AUC moyen: 0.7010 ± 0.0038
  F1 moyen: 0.0631 ± 0.0355
  Recall moyen: 0.0361 ± 0.0202
  Coût métier moyen: 1511.00 ± 34.85
  Seuil optimal: 0.50
🏃 View run V1_LogisticRegression_Baseline at: http://127.0.0.1:5000/#/experiments/1/runs/00e6a5708f0340678afb3fe611ba11c8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [6]:
# ============================================================================
# TABLEAU RÉCAPITULATIF: Métriques par fold
# ============================================================================

print("\n" + "="*110)
print("TABLEAU RÉCAPITULATIF: Métriques par fold")
print("="*110)

display_df = cv_results_df[["fold", "auc", "f1_score", "recall_class1", "business_cost_min", "optimal_threshold"]].copy()
print(display_df.to_string(index=False))

# Afficher les moyennes et écart-types
print("\n" + "="*110)
print("RÉSUMÉ GLOBAL: Moyennes et Écart-types sur 5 folds")
print("="*110)

summary_data = {
    "Métrique": ["AUC-ROC", "F1-Score", "Recall Classe 1", "Coût Métier Min", "Seuil Optimal"],
    "Moyenne": [
        f"{metrics_mean['auc']:.4f}",
        f"{metrics_mean['f1_score']:.4f}",
        f"{metrics_mean['recall_class1']:.4f}",
        f"{metrics_mean['business_cost_min']:.2f}",
        f"{metrics_mean['optimal_threshold']:.2f}",
    ],
    "Écart-type": [
        f"{metrics_std['auc']:.4f}",
        f"{metrics_std['f1_score']:.4f}",
        f"{metrics_std['recall_class1']:.4f}",
        f"{metrics_std['business_cost_min']:.2f}",
        "-",
    ]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print("="*110)



TABLEAU RÉCAPITULATIF: Métriques par fold
 fold      auc  f1_score  recall_class1  business_cost_min  optimal_threshold
    1 0.699467  0.056818       0.032258               1516                0.5
    2 0.697180  0.124294       0.070968               1451                0.5
    3 0.703589  0.056180       0.032258               1518                0.5
    4 0.706146  0.044444       0.025806               1531                0.5
    5 0.698425  0.033898       0.019355               1539                0.5

RÉSUMÉ GLOBAL: Moyennes et Écart-types sur 5 folds
       Métrique Moyenne Écart-type
        AUC-ROC  0.7010     0.0038
       F1-Score  0.0631     0.0355
Recall Classe 1  0.0361     0.0202
Coût Métier Min 1511.00      34.85
  Seuil Optimal    0.50          -


In [7]:
# ============================================================================
# ENTRAÎNEMENT FINAL: LogisticRegression sur l'ensemble train complet
# ============================================================================

final_model_v1 = LogisticRegression(**MODEL_CONFIG_V1)
final_model_v1.fit(X_train_scaled, y_train)

print("\n✓ Modèle final LogisticRegression V1 entraîné sur l'ensemble train complet")
print(f"  Nombre de features: {X_train_scaled.shape[1]}")
print(f"  Intercept: {final_model_v1.intercept_[0]:.6f}")
print(f"  Norme des coefficients: {np.linalg.norm(final_model_v1.coef_):.6f}")



✓ Modèle final LogisticRegression V1 entraîné sur l'ensemble train complet
  Nombre de features: 724
  Intercept: -3.504665
  Norme des coefficients: 3.948967


In [8]:
# ============================================================================
# ÉVALUATION SUR L'ENSEMBLE TEST
# ============================================================================
# Utiliser le même seuil fixe de 0.5

# Vérifier et nettoyer les NaN dans y_test
print(f"✓ Vérification initiale y_test:")
print(f"  y_test shape: {y_test.shape}")
print(f"  NaN in y_test: {y_test.isna().sum()}")

# Vérifier si y_test est entièrement NaN
if y_test.isna().sum() == len(y_test):
    print(f"\n⚠️  ATTENTION: y_test est entièrement NaN - Évaluation test IGNORÉE")
    print(f"   Les données de test n'ont pas de cible valide.")
    test_auc = None
    test_accuracy = None
    test_f1 = None
    test_recall = None
    test_cost = None
    tp_test = None
    fp_test = None
    fn_test = None
    tn_test = None
    
else:
    # Supprimer les lignes avec NaN dans y_test
    if y_test.isna().sum() > 0:
        print(f"  Suppression de {y_test.isna().sum()} lignes avec NaN dans y_test...")
        mask_test_clean = ~y_test.isna()
        y_test = y_test[mask_test_clean]
        X_test_scaled = X_test_scaled[mask_test_clean]
        print(f"  y_test après suppression: {y_test.shape}")
        print(f"  X_test_scaled après suppression: {X_test_scaled.shape}")

    # Réinitialiser les indices
    y_test.reset_index(drop=True, inplace=True)
    X_test_scaled.reset_index(drop=True, inplace=True)

    # Prédictions sur le test
    y_test_proba = final_model_v1.predict_proba(X_test_scaled)[:, 1]
    y_test_pred = (y_test_proba >= THRESHOLD_FIXED).astype(int)

    # Métriques sur le test
    test_auc = roc_auc_score(y_test, y_test_proba)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)

    # Coût métier
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_test_pred).ravel()
    test_cost = 10 * fn_test + 1 * fp_test

    print("\n" + "="*80)
    print("ÉVALUATION SUR ENSEMBLE TEST (seuil=0.5)")
    print("="*80)
    print(f"AUC-ROC:        {test_auc:.4f}")
    print(f"Accuracy:       {test_accuracy:.4f}")
    print(f"F1-Score:       {test_f1:.4f}")
    print(f"Recall Classe 1: {test_recall:.4f}")
    print(f"Coût Métier:    {test_cost:.0f}")
    print(f"\nConfusion Matrix:")
    print(f"  TP: {int(tp_test):6d}  |  FP: {int(fp_test):6d}")
    print(f"  FN: {int(fn_test):6d}  |  TN: {int(tn_test):6d}")
    print("="*80)


✓ Vérification initiale y_test:
  y_test shape: (10000,)
  NaN in y_test: 10000

⚠️  ATTENTION: y_test est entièrement NaN - Évaluation test IGNORÉE
   Les données de test n'ont pas de cible valide.


In [9]:
# ============================================================================
# LOGGING MLFLOW: Sauvegarde des métriques et artefacts du modèle
# ============================================================================
# Logger les métriques CV et le modèle comme artefact
# IMPORTANT: Le modèle n'est PAS enregistré dans la Model Registry automatiquement

# Terminer le run CV précédent
mlflow.end_run()

with mlflow.start_run(run_name="V1_LogisticRegression_Test_Evaluation"):
    # Logging des paramètres
    mlflow.log_params(MODEL_CONFIG_V1)
    
    # Tags
    mlflow.set_tag("version", "1")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("phase", "test_evaluation")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # TOUJOURS logger les métriques CV (pour que le modèle ait des métriques)
    mlflow.log_metric("auc", metrics_mean["auc"])
    mlflow.log_metric("f1_score", metrics_mean["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean["optimal_threshold"])
    
    # Si métriques test disponibles, les logger aussi (avec suffixe pour différencier)
    if test_auc is not None:
        mlflow.log_metric("test_auc", test_auc)
        mlflow.log_metric("test_f1_score", test_f1)
        mlflow.log_metric("test_recall_class1", test_recall)
        mlflow.log_metric("test_business_cost_min", test_cost)
        
        # Résultats test en artefact
        test_results = {
            "auc": float(test_auc),
            "f1_score": float(test_f1),
            "recall_class1": float(test_recall),
            "business_cost_min": float(test_cost),
            "optimal_threshold": float(THRESHOLD_FIXED),
            "confusion_matrix": {
                "tp": int(tp_test),
                "fp": int(fp_test),
                "fn": int(fn_test),
                "tn": int(tn_test),
            }
        }
        mlflow.log_dict(test_results, "test_evaluation.json")
        
        print(f"\n✓ Métriques test loggées")
        print(f"  Test AUC: {test_auc:.4f}")
        print(f"  Test F1: {test_f1:.4f}")
        print(f"  Test Recall: {test_recall:.4f}")
        print(f"  Test Business Cost Min: {test_cost:.0f}")
    else:
        print(f"\n⚠️  ATTENTION: Métriques test non disponibles (y_test était entièrement NaN)")
        print(f"   Les métriques CV sont utilisées.")
        mlflow.set_tag("test_metrics_available", "false")
    
    # LOG: Sauvegarder le modèle comme artefact (accessible via MLflow)
    # IMPORTANT: Le modèle n'est PAS enregistré dans la Model Registry automatiquement
    # Cela doit être fait manuellement via l'interface MLflow
    mlflow.sklearn.log_model(
        final_model_v1,
        artifact_path="logistic_regression_v1"
    )
    
    print(f"\n✓ Métriques et artefacts du modèle loggés dans MLflow")
    print(f"  AUC (CV): {metrics_mean['auc']:.4f}")
    print(f"  F1 (CV): {metrics_mean['f1_score']:.4f}")
    print(f"  Recall (CV): {metrics_mean['recall_class1']:.4f}")
    print(f"  Business Cost Min (CV): {metrics_mean['business_cost_min']:.2f}")
    print(f"\n  ℹ️  Pour enregistrer le modèle dans la Model Registry :")
    print(f"     - Allez à http://127.0.0.1:5000/#/experiments/1")
    print(f"     - Trouvez le run 'V1_LogisticRegression_Test_Evaluation'")
    print(f"     - Dans l'onglet 'Artifacts', cliquez 'Register Model'")
    print(f"     - Sélectionnez ou créez le nom 'LogisticRegression_V1'")


2026/02/06 01:40:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



⚠️  ATTENTION: Métriques test non disponibles (y_test était entièrement NaN)
   Les métriques CV sont utilisées.


2026/02/06 01:40:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



✓ Métriques et artefacts du modèle loggés dans MLflow
  AUC (CV): 0.7010
  F1 (CV): 0.0631
  Recall (CV): 0.0361
  Business Cost Min (CV): 1511.00

  ℹ️  Pour enregistrer le modèle dans la Model Registry :
     - Allez à http://127.0.0.1:5000/#/experiments/1
     - Trouvez le run 'V1_LogisticRegression_Test_Evaluation'
     - Dans l'onglet 'Artifacts', cliquez 'Register Model'
     - Sélectionnez ou créez le nom 'LogisticRegression_V1'
🏃 View run V1_LogisticRegression_Test_Evaluation at: http://127.0.0.1:5000/#/experiments/1/runs/b98cbeb8fddc435f998b929565c06021
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [10]:
# ============================================================================
# COMPARAISON: Cross-Validation vs Test
# ============================================================================
# Vérifier la stabilité du modèle (généralisation)

print("\n" + "="*100)
print("COMPARAISON: Cross-Validation vs Test Set")
print("="*100)

if test_auc is None:
    print("\n⚠️  ATTENTION: Métriques test non disponibles")
    print("   (y_test était entièrement NaN - Évaluation test ignorée)")
    print("\n   Affichage des métriques de Cross-Validation uniquement:")
    print(f"   AUC moyen: {metrics_mean['auc']:.4f} ± {metrics_std['auc']:.4f}")
    print(f"   F1 moyen: {metrics_mean['f1_score']:.4f} ± {metrics_std['f1_score']:.4f}")
    print(f"   Recall moyen: {metrics_mean['recall_class1']:.4f} ± {metrics_std['recall_class1']:.4f}")
    print(f"   Coût métier moyen: {metrics_mean['business_cost_min']:.2f} ± {metrics_std['business_cost_min']:.2f}")
    print(f"   Seuil optimal: {metrics_mean['optimal_threshold']:.2f}")
else:
    comparison_data = {
        "Métrique": ["AUC-ROC", "F1-Score", "Recall Classe 1", "Coût Métier Min", "Seuil Optimal"],
        "CV Mean": [
            f"{metrics_mean['auc']:.4f}",
            f"{metrics_mean['f1_score']:.4f}",
            f"{metrics_mean['recall_class1']:.4f}",
            f"{metrics_mean['business_cost_min']:.2f}",
            f"{metrics_mean['optimal_threshold']:.2f}",
        ],
        "Test": [
            f"{test_auc:.4f}",
            f"{test_f1:.4f}",
            f"{test_recall:.4f}",
            f"{test_cost:.2f}",
            f"{THRESHOLD_FIXED:.2f}",
        ],
        "Diff (Test-CV)": [
            f"{test_auc - metrics_mean['auc']:+.4f}",
            f"{test_f1 - metrics_mean['f1_score']:+.4f}",
            f"{test_recall - metrics_mean['recall_class1']:+.4f}",
            f"{test_cost - metrics_mean['business_cost_min']:+.2f}",
            "0.00",
        ]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

print("="*100)



COMPARAISON: Cross-Validation vs Test Set

⚠️  ATTENTION: Métriques test non disponibles
   (y_test était entièrement NaN - Évaluation test ignorée)

   Affichage des métriques de Cross-Validation uniquement:
   AUC moyen: 0.7010 ± 0.0038
   F1 moyen: 0.0631 ± 0.0355
   Recall moyen: 0.0361 ± 0.0202
   Coût métier moyen: 1511.00 ± 34.85
   Seuil optimal: 0.50


In [11]:
# ============================================================================
# VERSION 2.1: LogisticRegression avec class_weight='balanced'
# ============================================================================
# Objectif: Gérer le déséquilibre des classes avec class_weight='balanced'
# Validation: StratifiedKFold (5 folds)
# Modèle: LogisticRegression(max_iter=1000, random_state=42, solver='saga', class_weight='balanced', penalty='l2')
# Features: X_train_scaled, y_train (déjà scalées)
# Seuil fixe: 0.5
# Métriques par fold: AUC-ROC, Accuracy, F1-score, Recall classe 1
# Coût métier: 10 * FN + 1 * FP (avec seuil=0.5)
# MLflow: run_name="V2_LogisticRegression_ClassWeightBalanced"
# Tags: version="2", imbalance_handling="class_weight"

from sklearn.pipeline import Pipeline

# Configuration du modèle V2.1 (class_weight balanced)
MODEL_CONFIG_V2_1 = {
    "max_iter": 3000,
    "random_state": 42,
    "solver": "saga",
    "class_weight": "balanced"
}

RUN_NAME_V2_1 = "V2_LogisticRegression_ClassWeightBalanced"

fold_results_v2_1 = []

# Terminer tout run actif
mlflow.end_run()

with mlflow.start_run(run_name=RUN_NAME_V2_1):
    # ========== Logging des paramètres et tags ==========
    mlflow.log_params(MODEL_CONFIG_V2_1)
    mlflow.set_tag("version", "2")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("notebook", NOTEBOOK_NAME)
    mlflow.set_tag("phase", "imbalance_handling_cv")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("scaling", "StandardScaler")
    mlflow.set_tag("imbalance_handling", "class_weight")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # ========== StratifiedKFold (5 folds) ==========
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train_scaled, y_train), start=1):
        X_tr, X_val = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # ========== Entraînement ==========
        model = LogisticRegression(**MODEL_CONFIG_V2_1)
        model.fit(X_tr, y_tr)
        
        # ========== Prédictions ==========
        y_val_proba = model.predict_proba(X_val)[:, 1]
        y_val_pred = (y_val_proba >= THRESHOLD_FIXED).astype(int)
        
        # ========== Métriques ==========
        auc = roc_auc_score(y_val, y_val_proba)
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        
        # ========== Coût métier (seuil=0.5) ==========
        tn, fp, fn, tp = confusion_matrix(y_val, y_val_pred).ravel()
        cost = 10 * fn + 1 * fp
        
        fold_results_v2_1.append({
            "fold": fold_idx,
            "auc": auc,
            "accuracy": accuracy,
            "f1_score": f1,
            "recall_class1": recall,
            "business_cost_min": cost,
            "optimal_threshold": THRESHOLD_FIXED,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn
        })
        
        print(f"Fold {fold_idx}/5 | AUC={auc:.4f} | Acc={accuracy:.4f} | "
              f"F1={f1:.4f} | Recall={recall:.4f} | Cost={cost:.0f}")
    
    # ========== Agrégation des résultats ==========
    cv_results_v2_1_df = pd.DataFrame(fold_results_v2_1)
    
    metrics_mean_v2_1 = {
        "auc": cv_results_v2_1_df["auc"].mean(),
        "f1_score": cv_results_v2_1_df["f1_score"].mean(),
        "recall_class1": cv_results_v2_1_df["recall_class1"].mean(),
        "business_cost_min": cv_results_v2_1_df["business_cost_min"].mean(),
        "optimal_threshold": THRESHOLD_FIXED,
    }
    
    metrics_std_v2_1 = {
        "auc": cv_results_v2_1_df["auc"].std(),
        "f1_score": cv_results_v2_1_df["f1_score"].std(),
        "recall_class1": cv_results_v2_1_df["recall_class1"].std(),
        "business_cost_min": cv_results_v2_1_df["business_cost_min"].std(),
    }
    
    # ========== Logging dans MLFlow ==========
    mlflow.log_metric("auc", metrics_mean_v2_1["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v2_1["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v2_1["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v2_1["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v2_1["optimal_threshold"])
    
    # Log artefact JSON avec détails par fold
    mlflow.log_dict(cv_results_v2_1_df.to_dict(orient="records"), "cv_results_per_fold.json")
    
    print("\n✓ Cross-Validation LogisticRegression V2.1 (class_weight='balanced') terminée")
    print(f"  AUC moyen: {metrics_mean_v2_1['auc']:.4f} ± {metrics_std_v2_1['auc']:.4f}")
    print(f"  F1 moyen: {metrics_mean_v2_1['f1_score']:.4f} ± {metrics_std_v2_1['f1_score']:.4f}")
    print(f"  Recall moyen: {metrics_mean_v2_1['recall_class1']:.4f} ± {metrics_std_v2_1['recall_class1']:.4f}")
    print(f"  Coût métier moyen: {metrics_mean_v2_1['business_cost_min']:.2f} ± {metrics_std_v2_1['business_cost_min']:.2f}")
    print(f"  Seuil optimal: {metrics_mean_v2_1['optimal_threshold']:.2f}")

Fold 1/5 | AUC=0.6886 | Acc=0.6995 | F1=0.2324 | Recall=0.5871 | Cost=1177
Fold 2/5 | AUC=0.6828 | Acc=0.6980 | F1=0.2412 | Recall=0.6194 | Cost=1135
Fold 3/5 | AUC=0.7118 | Acc=0.7265 | F1=0.2476 | Recall=0.5806 | Cost=1132
Fold 4/5 | AUC=0.7035 | Acc=0.7000 | F1=0.2347 | Recall=0.5935 | Cost=1167
Fold 5/5 | AUC=0.6920 | Acc=0.7185 | F1=0.2277 | Recall=0.5355 | Cost=1211

✓ Cross-Validation LogisticRegression V2.1 (class_weight='balanced') terminée
  AUC moyen: 0.6957 ± 0.0117
  F1 moyen: 0.2367 ± 0.0078
  Recall moyen: 0.5832 ± 0.0305
  Coût métier moyen: 1164.40 ± 32.60
  Seuil optimal: 0.50
🏃 View run V2_LogisticRegression_ClassWeightBalanced at: http://127.0.0.1:5000/#/experiments/1/runs/d8b12c8475984c75b995472e30f56f69
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [12]:
# ============================================================================
# ENTRAÎNEMENT FINAL V2.1: LogisticRegression avec class_weight='balanced'
# ============================================================================

final_model_v2_1 = LogisticRegression(**MODEL_CONFIG_V2_1)
final_model_v2_1.fit(X_train_scaled, y_train)

print("\n✓ Modèle final LogisticRegression V2.1 entraîné sur l'ensemble train complet")
print(f"  Nombre de features: {X_train_scaled.shape[1]}")
print(f"  Intercept: {final_model_v2_1.intercept_[0]:.6f}")
print(f"  Norme des coefficients: {np.linalg.norm(final_model_v2_1.coef_):.6f}")

# ============================================================================
# LOGGING MLFLOW V2.1: Sauvegarde du modèle
# ============================================================================

mlflow.end_run()

with mlflow.start_run(run_name="V2.1_LogisticRegression_ClassWeight_Final"):
    # Logging des paramètres
    mlflow.log_params(MODEL_CONFIG_V2_1)
    
    # Tags
    mlflow.set_tag("version", "2.1")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("phase", "final_model")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("imbalance_handling", "class_weight")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # Logger les métriques CV
    mlflow.log_metric("auc", metrics_mean_v2_1["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v2_1["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v2_1["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v2_1["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v2_1["optimal_threshold"])
    
    # Sauvegarder le modèle comme artefact
    mlflow.sklearn.log_model(
        final_model_v2_1,
        artifact_path="logistic_regression_v2_1_class_weight"
    )
    
    print(f"\n✓ Modèle V2.1 enregistré dans MLflow")
    print(f"  AUC (CV): {metrics_mean_v2_1['auc']:.4f}")
    print(f"  F1 (CV): {metrics_mean_v2_1['f1_score']:.4f}")
    print(f"  Recall (CV): {metrics_mean_v2_1['recall_class1']:.4f}")
    print(f"  Business Cost Min (CV): {metrics_mean_v2_1['business_cost_min']:.2f}")


✓ Modèle final LogisticRegression V2.1 entraîné sur l'ensemble train complet
  Nombre de features: 724
  Intercept: -0.917467
  Norme des coefficients: 2.441323


2026/02/06 01:48:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/06 01:48:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



✓ Modèle V2.1 enregistré dans MLflow
  AUC (CV): 0.6957
  F1 (CV): 0.2367
  Recall (CV): 0.5832
  Business Cost Min (CV): 1164.40
🏃 View run V2.1_LogisticRegression_ClassWeight_Final at: http://127.0.0.1:5000/#/experiments/1/runs/0bc8f5f187c94a349c72011de4524c77
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [13]:
# ============================================================================
# VERSION 2.2: LogisticRegression avec SMOTE
# ============================================================================
# Objectif: Gérer le déséquilibre des classes avec SMOTE
# Validation: StratifiedKFold (5 folds)
# Modèle: LogisticRegression(max_iter=1000, random_state=42, solver='saga', penalty='l2')
# Pipeline: StandardScaler -> SMOTE -> LogisticRegression (pour éviter le data leakage)
# Features: X_train, y_train (seront scalées dans le pipeline)
# Seuil fixe: 0.5
# Métriques par fold: AUC-ROC, Accuracy, F1-score, Recall classe 1
# Coût métier: 10 * FN + 1 * FP (avec seuil=0.5)
# MLflow: run_name="V2_LogisticRegression_SMOTE"
# Tags: version="2", imbalance_handling="smote"

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Configuration du modèle V2.2 (SMOTE)
MODEL_CONFIG_V2_2 = {
    "max_iter": 3000,
    "random_state": 42,
    "solver": "saga"
}

RUN_NAME_V2_2 = "V2_LogisticRegression_SMOTE"

fold_results_v2_2 = []

# Terminer tout run actif
mlflow.end_run()

with mlflow.start_run(run_name=RUN_NAME_V2_2):
    # ========== Logging des paramètres et tags ==========
    mlflow.log_params(MODEL_CONFIG_V2_2)
    mlflow.set_tag("version", "2")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("notebook", NOTEBOOK_NAME)
    mlflow.set_tag("phase", "imbalance_handling_cv")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("scaling", "StandardScaler")
    mlflow.set_tag("imbalance_handling", "smote")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # ========== StratifiedKFold (5 folds) ==========
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), start=1):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # ========== Pipeline: Scaler -> SMOTE -> Model ==========
        # SMOTE est appliqué uniquement sur le train de chaque fold
        pipeline = ImbPipeline([
            ('scaler', StandardScaler()),
            ('smote', SMOTE(random_state=RANDOM_STATE)),
            ('model', LogisticRegression(**MODEL_CONFIG_V2_2))
        ])
        
        # ========== Entraînement ==========
        pipeline.fit(X_tr, y_tr)
        
        # ========== Prédictions ==========
        y_val_proba = pipeline.predict_proba(X_val)[:, 1]
        y_val_pred = (y_val_proba >= THRESHOLD_FIXED).astype(int)
        
        # ========== Métriques ==========
        auc = roc_auc_score(y_val, y_val_proba)
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        
        # ========== Coût métier (seuil=0.5) ==========
        tn, fp, fn, tp = confusion_matrix(y_val, y_val_pred).ravel()
        cost = 10 * fn + 1 * fp
        
        fold_results_v2_2.append({
            "fold": fold_idx,
            "auc": auc,
            "accuracy": accuracy,
            "f1_score": f1,
            "recall_class1": recall,
            "business_cost_min": cost,
            "optimal_threshold": THRESHOLD_FIXED,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn
        })
        
        print(f"Fold {fold_idx}/5 | AUC={auc:.4f} | Acc={accuracy:.4f} | "
              f"F1={f1:.4f} | Recall={recall:.4f} | Cost={cost:.0f}")
    
    # ========== Agrégation des résultats ==========
    cv_results_v2_2_df = pd.DataFrame(fold_results_v2_2)
    
    metrics_mean_v2_2 = {
        "auc": cv_results_v2_2_df["auc"].mean(),
        "f1_score": cv_results_v2_2_df["f1_score"].mean(),
        "recall_class1": cv_results_v2_2_df["recall_class1"].mean(),
        "business_cost_min": cv_results_v2_2_df["business_cost_min"].mean(),
        "optimal_threshold": THRESHOLD_FIXED,
    }
    
    metrics_std_v2_2 = {
        "auc": cv_results_v2_2_df["auc"].std(),
        "f1_score": cv_results_v2_2_df["f1_score"].std(),
        "recall_class1": cv_results_v2_2_df["recall_class1"].std(),
        "business_cost_min": cv_results_v2_2_df["business_cost_min"].std(),
    }
    
    # ========== Logging dans MLFlow ==========
    mlflow.log_metric("auc", metrics_mean_v2_2["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v2_2["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v2_2["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v2_2["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v2_2["optimal_threshold"])
    
    # Log artefact JSON avec détails par fold
    mlflow.log_dict(cv_results_v2_2_df.to_dict(orient="records"), "cv_results_per_fold.json")
    
    print("\n✓ Cross-Validation LogisticRegression V2.2 (SMOTE) terminée")
    print(f"  AUC moyen: {metrics_mean_v2_2['auc']:.4f} ± {metrics_std_v2_2['auc']:.4f}")
    print(f"  F1 moyen: {metrics_mean_v2_2['f1_score']:.4f} ± {metrics_std_v2_2['f1_score']:.4f}")
    print(f"  Recall moyen: {metrics_mean_v2_2['recall_class1']:.4f} ± {metrics_std_v2_2['recall_class1']:.4f}")
    print(f"  Coût métier moyen: {metrics_mean_v2_2['business_cost_min']:.2f} ± {metrics_std_v2_2['business_cost_min']:.2f}")
    print(f"  Seuil optimal: {metrics_mean_v2_2['optimal_threshold']:.2f}")

Fold 1/5 | AUC=0.6801 | Acc=0.7145 | F1=0.2397 | Recall=0.5806 | Cost=1156
Fold 2/5 | AUC=0.6807 | Acc=0.6985 | F1=0.2299 | Recall=0.5806 | Cost=1188
Fold 3/5 | AUC=0.7055 | Acc=0.7375 | F1=0.2553 | Recall=0.5806 | Cost=1110
Fold 4/5 | AUC=0.6872 | Acc=0.7190 | F1=0.2301 | Recall=0.5419 | Cost=1201
Fold 5/5 | AUC=0.6914 | Acc=0.7435 | F1=0.2377 | Recall=0.5161 | Cost=1188

✓ Cross-Validation LogisticRegression V2.2 (SMOTE) terminée
  AUC moyen: 0.6890 ± 0.0104
  F1 moyen: 0.2386 ± 0.0104
  Recall moyen: 0.5600 ± 0.0297
  Coût métier moyen: 1168.60 ± 36.73
  Seuil optimal: 0.50
🏃 View run V2_LogisticRegression_SMOTE at: http://127.0.0.1:5000/#/experiments/1/runs/dab29ff5c5a14880bb75287b1c5bcd5c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [14]:
# ============================================================================
# ENTRAÎNEMENT FINAL V2.2: LogisticRegression avec SMOTE
# ============================================================================

# Pipeline complet avec SMOTE sur l'ensemble train
final_pipeline_v2_2 = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('model', LogisticRegression(**MODEL_CONFIG_V2_2))
])

final_pipeline_v2_2.fit(X_train, y_train)

print("\n✓ Pipeline final LogisticRegression V2.2 (SMOTE) entraîné sur l'ensemble train complet")
print(f"  Nombre de features: {X_train.shape[1]}")
print(f"  Intercept: {final_pipeline_v2_2.named_steps['model'].intercept_[0]:.6f}")
print(f"  Norme des coefficients: {np.linalg.norm(final_pipeline_v2_2.named_steps['model'].coef_):.6f}")

# ============================================================================
# LOGGING MLFLOW V2.2: Sauvegarde du modèle
# ============================================================================

mlflow.end_run()

with mlflow.start_run(run_name="V2.2_LogisticRegression_SMOTE_Final"):
    # Logging des paramètres
    mlflow.log_params(MODEL_CONFIG_V2_2)
    
    # Tags
    mlflow.set_tag("version", "2.2")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("phase", "final_model")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("imbalance_handling", "smote")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # Logger les métriques CV
    mlflow.log_metric("auc", metrics_mean_v2_2["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v2_2["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v2_2["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v2_2["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v2_2["optimal_threshold"])
    
    # Sauvegarder le pipeline complet comme artefact
    mlflow.sklearn.log_model(
        final_pipeline_v2_2,
        artifact_path="logistic_regression_v2_2_smote"
    )
    
    print(f"\n✓ Pipeline V2.2 enregistré dans MLflow")
    print(f"  AUC (CV): {metrics_mean_v2_2['auc']:.4f}")
    print(f"  F1 (CV): {metrics_mean_v2_2['f1_score']:.4f}")
    print(f"  Recall (CV): {metrics_mean_v2_2['recall_class1']:.4f}")
    print(f"  Business Cost Min (CV): {metrics_mean_v2_2['business_cost_min']:.2f}")


✓ Pipeline final LogisticRegression V2.2 (SMOTE) entraîné sur l'ensemble train complet
  Nombre de features: 724
  Intercept: -1.226644
  Norme des coefficients: 3.213375


2026/02/06 02:01:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/06 02:01:59 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



✓ Pipeline V2.2 enregistré dans MLflow
  AUC (CV): 0.6890
  F1 (CV): 0.2386
  Recall (CV): 0.5600
  Business Cost Min (CV): 1168.60
🏃 View run V2.2_LogisticRegression_SMOTE_Final at: http://127.0.0.1:5000/#/experiments/1/runs/9a1cd90834c84f43a6b660e9dcc0a408
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [15]:
# ============================================================================
# TABLEAU COMPARATIF: V1 Baseline vs V2 Class Weight vs V2 SMOTE
# ============================================================================

print("\n" + "="*120)
print("TABLEAU COMPARATIF: V1 Baseline vs V2.1 Class Weight vs V2.2 SMOTE")
print("="*120)

comparison_data = {
    "Version": ["V1 Baseline", "V2.1 Class Weight", "V2.2 SMOTE"],
    "AUC": [
        f"{metrics_mean['auc']:.4f} ± {metrics_std['auc']:.4f}",
        f"{metrics_mean_v2_1['auc']:.4f} ± {metrics_std_v2_1['auc']:.4f}",
        f"{metrics_mean_v2_2['auc']:.4f} ± {metrics_std_v2_2['auc']:.4f}",
    ],
    "F1-Score": [
        f"{metrics_mean['f1_score']:.4f} ± {metrics_std['f1_score']:.4f}",
        f"{metrics_mean_v2_1['f1_score']:.4f} ± {metrics_std_v2_1['f1_score']:.4f}",
        f"{metrics_mean_v2_2['f1_score']:.4f} ± {metrics_std_v2_2['f1_score']:.4f}",
    ],
    "Recall Classe 1": [
        f"{metrics_mean['recall_class1']:.4f} ± {metrics_std['recall_class1']:.4f}",
        f"{metrics_mean_v2_1['recall_class1']:.4f} ± {metrics_std_v2_1['recall_class1']:.4f}",
        f"{metrics_mean_v2_2['recall_class1']:.4f} ± {metrics_std_v2_2['recall_class1']:.4f}",
    ],
    "Coût Métier Min": [
        f"{metrics_mean['business_cost_min']:.2f} ± {metrics_std['business_cost_min']:.2f}",
        f"{metrics_mean_v2_1['business_cost_min']:.2f} ± {metrics_std_v2_1['business_cost_min']:.2f}",
        f"{metrics_mean_v2_2['business_cost_min']:.2f} ± {metrics_std_v2_2['business_cost_min']:.2f}",
    ],
    "Imbalance Handling": [
        "None",
        "class_weight='balanced'",
        "SMOTE",
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))
print("="*120)

# Analyse des améliorations
print("\n" + "="*120)
print("ANALYSE DES AMÉLIORATIONS (vs V1 Baseline)")
print("="*120)

improvement_data = {
    "Version": ["V2.1 Class Weight", "V2.2 SMOTE"],
    "Δ AUC": [
        f"{metrics_mean_v2_1['auc'] - metrics_mean['auc']:+.4f}",
        f"{metrics_mean_v2_2['auc'] - metrics_mean['auc']:+.4f}",
    ],
    "Δ F1-Score": [
        f"{metrics_mean_v2_1['f1_score'] - metrics_mean['f1_score']:+.4f}",
        f"{metrics_mean_v2_2['f1_score'] - metrics_mean['f1_score']:+.4f}",
    ],
    "Δ Recall Classe 1": [
        f"{metrics_mean_v2_1['recall_class1'] - metrics_mean['recall_class1']:+.4f}",
        f"{metrics_mean_v2_2['recall_class1'] - metrics_mean['recall_class1']:+.4f}",
    ],
    "Δ Coût Métier": [
        f"{metrics_mean_v2_1['business_cost_min'] - metrics_mean['business_cost_min']:+.2f}",
        f"{metrics_mean_v2_2['business_cost_min'] - metrics_mean['business_cost_min']:+.2f}",
    ]
}

improvement_df = pd.DataFrame(improvement_data)
print(improvement_df.to_string(index=False))
print("="*120)

# Déterminer la meilleure version
best_auc_version = ["V1", "V2.1", "V2.2"][
    np.argmax([metrics_mean['auc'], metrics_mean_v2_1['auc'], metrics_mean_v2_2['auc']])
]
best_f1_version = ["V1", "V2.1", "V2.2"][
    np.argmax([metrics_mean['f1_score'], metrics_mean_v2_1['f1_score'], metrics_mean_v2_2['f1_score']])
]
best_recall_version = ["V1", "V2.1", "V2.2"][
    np.argmax([metrics_mean['recall_class1'], metrics_mean_v2_1['recall_class1'], metrics_mean_v2_2['recall_class1']])
]
best_cost_version = ["V1", "V2.1", "V2.2"][
    np.argmin([metrics_mean['business_cost_min'], metrics_mean_v2_1['business_cost_min'], metrics_mean_v2_2['business_cost_min']])
]

print("\n✓ Meilleure version par métrique:")
print(f"  AUC:              {best_auc_version}")
print(f"  F1-Score:         {best_f1_version}")
print(f"  Recall Classe 1:  {best_recall_version}")
print(f"  Coût Métier Min:  {best_cost_version}")
print("="*120)


TABLEAU COMPARATIF: V1 Baseline vs V2.1 Class Weight vs V2.2 SMOTE
          Version             AUC        F1-Score Recall Classe 1 Coût Métier Min      Imbalance Handling
      V1 Baseline 0.7010 ± 0.0038 0.0631 ± 0.0355 0.0361 ± 0.0202 1511.00 ± 34.85                    None
V2.1 Class Weight 0.6957 ± 0.0117 0.2367 ± 0.0078 0.5832 ± 0.0305 1164.40 ± 32.60 class_weight='balanced'
       V2.2 SMOTE 0.6890 ± 0.0104 0.2386 ± 0.0104 0.5600 ± 0.0297 1168.60 ± 36.73                   SMOTE

ANALYSE DES AMÉLIORATIONS (vs V1 Baseline)
          Version   Δ AUC Δ F1-Score Δ Recall Classe 1 Δ Coût Métier
V2.1 Class Weight -0.0052    +0.1736           +0.5471       -346.60
       V2.2 SMOTE -0.0120    +0.1754           +0.5239       -342.40

✓ Meilleure version par métrique:
  AUC:              V1
  F1-Score:         V2.2
  Recall Classe 1:  V2.1
  Coût Métier Min:  V2.1


# VERSION 3: Meilleur modèle avec scaling robuste optimisé

Objectif: Réentraîner le meilleur modèle (V2.1 class_weight='balanced') avec un scaling plus adapté
- **RobustScaler**: Utilise la médiane et l'IQR (moins sensible aux outliers que StandardScaler)
- Validation: StratifiedKFold (5 folds)
- Modèle: LogisticRegression avec class_weight='balanced'
- Enregistrement dans MLflow Model Registry sous le nom "regression"

In [17]:
# ============================================================================
# VERSION 3: Meilleur modèle avec RobustScaler (adapté aux outliers)
# ============================================================================
# Objectif: Améliorer le scaling pour des features avec outliers
# RobustScaler utilise la médiane et l'IQR au lieu de la moyenne et l'écart-type
# Plus robuste face aux valeurs extrêmes dans les données de crédit

from sklearn.preprocessing import RobustScaler

# Créer le RobustScaler
robust_scaler = RobustScaler()
X_train_robust = robust_scaler.fit_transform(X_train)
X_test_robust = robust_scaler.transform(X_test)

# Reconvertir en DataFrame
X_train_robust = pd.DataFrame(X_train_robust, columns=X_train.columns)
X_test_robust = pd.DataFrame(X_test_robust, columns=X_test.columns)

print(f"\n✓ Features scalées avec RobustScaler (médiane + IQR):")
print(f"  Shape train: {X_train_robust.shape}")
print(f"  Median: {X_train_robust.median().mean():.8f} (≈ 0)")
print(f"  IQR (Interquartile Range): {(X_train_robust.quantile(0.75) - X_train_robust.quantile(0.25)).mean():.6f}")
print(f"\n  Comparaison vs StandardScaler:")
print(f"  StandardScaler - Mean: {X_train_scaled.mean().mean():.8f}, Std: {X_train_scaled.std().mean():.6f}")
print(f"  RobustScaler   - Median: {X_train_robust.median().mean():.8f}, IQR: {(X_train_robust.quantile(0.75) - X_train_robust.quantile(0.25)).mean():.6f}")


✓ Features scalées avec RobustScaler (médiane + IQR):
  Shape train: (10000, 724)
  Median: 0.00000000 (≈ 0)
  IQR (Interquartile Range): 0.052486

  Comparaison vs StandardScaler:
  StandardScaler - Mean: -0.00000000, Std: 0.874353
  RobustScaler   - Median: 0.00000000, IQR: 0.052486


In [18]:
# ============================================================================
# CROSS-VALIDATION V3: Meilleur modèle (V2.1) avec RobustScaler
# ============================================================================

MODEL_CONFIG_V3 = {
    "max_iter": 3000,
    "random_state": 42,
    "solver": "saga",
    "class_weight": "balanced"
}

RUN_NAME_V3 = "V3_LogisticRegression_RobustScaler_ClassWeight"

fold_results_v3 = []

# Terminer tout run actif
mlflow.end_run()

with mlflow.start_run(run_name=RUN_NAME_V3):
    # ========== Logging des paramètres et tags ==========
    mlflow.log_params(MODEL_CONFIG_V3)
    mlflow.set_tag("version", "3")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("notebook", NOTEBOOK_NAME)
    mlflow.set_tag("phase", "robust_scaling_cv")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("scaling", "RobustScaler")
    mlflow.set_tag("imbalance_handling", "class_weight")
    mlflow.set_tag("model_type", "LogisticRegression")
    
    # ========== StratifiedKFold (5 folds) ==========
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_train_robust, y_train), start=1):
        X_tr, X_val = X_train_robust.iloc[train_idx], X_train_robust.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # ========== Entraînement ==========
        model = LogisticRegression(**MODEL_CONFIG_V3)
        model.fit(X_tr, y_tr)
        
        # ========== Prédictions ==========
        y_val_proba = model.predict_proba(X_val)[:, 1]
        y_val_pred = (y_val_proba >= THRESHOLD_FIXED).astype(int)
        
        # ========== Métriques ==========
        auc = roc_auc_score(y_val, y_val_proba)
        accuracy = accuracy_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)
        recall = recall_score(y_val, y_val_pred)
        
        # ========== Coût métier (seuil=0.5) ==========
        tn, fp, fn, tp = confusion_matrix(y_val, y_val_pred).ravel()
        cost = 10 * fn + 1 * fp
        
        fold_results_v3.append({
            "fold": fold_idx,
            "auc": auc,
            "accuracy": accuracy,
            "f1_score": f1,
            "recall_class1": recall,
            "business_cost_min": cost,
            "optimal_threshold": THRESHOLD_FIXED,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn
        })
        
        print(f"Fold {fold_idx}/5 | AUC={auc:.4f} | Acc={accuracy:.4f} | "
              f"F1={f1:.4f} | Recall={recall:.4f} | Cost={cost:.0f}")
    
    # ========== Agrégation des résultats ==========
    cv_results_v3_df = pd.DataFrame(fold_results_v3)
    
    metrics_mean_v3 = {
        "auc": cv_results_v3_df["auc"].mean(),
        "f1_score": cv_results_v3_df["f1_score"].mean(),
        "recall_class1": cv_results_v3_df["recall_class1"].mean(),
        "business_cost_min": cv_results_v3_df["business_cost_min"].mean(),
        "optimal_threshold": THRESHOLD_FIXED,
    }
    
    metrics_std_v3 = {
        "auc": cv_results_v3_df["auc"].std(),
        "f1_score": cv_results_v3_df["f1_score"].std(),
        "recall_class1": cv_results_v3_df["recall_class1"].std(),
        "business_cost_min": cv_results_v3_df["business_cost_min"].std(),
    }
    
    # ========== Logging dans MLFlow ==========
    mlflow.log_metric("auc", metrics_mean_v3["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v3["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v3["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v3["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v3["optimal_threshold"])
    
    # Log artefact JSON avec détails par fold
    mlflow.log_dict(cv_results_v3_df.to_dict(orient="records"), "cv_results_per_fold.json")
    
    print("\n✓ Cross-Validation LogisticRegression V3 (RobustScaler + class_weight) terminée")
    print(f"  AUC moyen: {metrics_mean_v3['auc']:.4f} ± {metrics_std_v3['auc']:.4f}")
    print(f"  F1 moyen: {metrics_mean_v3['f1_score']:.4f} ± {metrics_std_v3['f1_score']:.4f}")
    print(f"  Recall moyen: {metrics_mean_v3['recall_class1']:.4f} ± {metrics_std_v3['recall_class1']:.4f}")
    print(f"  Coût métier moyen: {metrics_mean_v3['business_cost_min']:.2f} ± {metrics_std_v3['business_cost_min']:.2f}")
    print(f"  Seuil optimal: {metrics_mean_v3['optimal_threshold']:.2f}")

Fold 1/5 | AUC=0.5488 | Acc=0.3675 | F1=0.1516 | Recall=0.7290 | Cost=1643
Fold 2/5 | AUC=0.5648 | Acc=0.4400 | F1=0.1592 | Recall=0.6839 | Cost=1561
Fold 3/5 | AUC=0.5284 | Acc=0.3270 | F1=0.1492 | Recall=0.7613 | Cost=1679
Fold 4/5 | AUC=0.5628 | Acc=0.3750 | F1=0.1554 | Recall=0.7419 | Cost=1610
Fold 5/5 | AUC=0.5070 | Acc=0.3575 | F1=0.1462 | Recall=0.7097 | Cost=1690

✓ Cross-Validation LogisticRegression V3 (RobustScaler + class_weight) terminée
  AUC moyen: 0.5424 ± 0.0245
  F1 moyen: 0.1523 ± 0.0051
  Recall moyen: 0.7252 ± 0.0298
  Coût métier moyen: 1636.60 ± 52.71
  Seuil optimal: 0.50
🏃 View run V3_LogisticRegression_RobustScaler_ClassWeight at: http://127.0.0.1:5000/#/experiments/1/runs/f3c1d8a8220a4e5193cba3eb73b30df6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [19]:
# ============================================================================
# ENTRAÎNEMENT FINAL V3 + ENREGISTREMENT DANS MODEL REGISTRY
# ============================================================================

# Entraîner le modèle final sur tout le train set avec RobustScaler
final_model_v3 = LogisticRegression(**MODEL_CONFIG_V3)
final_model_v3.fit(X_train_robust, y_train)

print("\n✓ Modèle final LogisticRegression V3 entraîné sur l'ensemble train complet")
print(f"  Nombre de features: {X_train_robust.shape[1]}")
print(f"  Intercept: {final_model_v3.intercept_[0]:.6f}")
print(f"  Norme des coefficients: {np.linalg.norm(final_model_v3.coef_):.6f}")

# ============================================================================
# LOGGING MLFLOW V3 + ENREGISTREMENT DANS MODEL REGISTRY
# ============================================================================

mlflow.end_run()

with mlflow.start_run(run_name="V3_LogisticRegression_RobustScaler_Final"):
    # Logging des paramètres
    mlflow.log_params(MODEL_CONFIG_V3)
    
    # Tags
    mlflow.set_tag("version", "3")
    mlflow.set_tag("model", "LogisticRegression")
    mlflow.set_tag("phase", "final_model")
    mlflow.set_tag("threshold", str(THRESHOLD_FIXED))
    mlflow.set_tag("scaling", "RobustScaler")
    mlflow.set_tag("imbalance_handling", "class_weight")
    mlflow.set_tag("model_type", "LogisticRegression")
    mlflow.set_tag("best_model", "true")
    
    # Logger les métriques CV
    mlflow.log_metric("auc", metrics_mean_v3["auc"])
    mlflow.log_metric("f1_score", metrics_mean_v3["f1_score"])
    mlflow.log_metric("recall_class1", metrics_mean_v3["recall_class1"])
    mlflow.log_metric("business_cost_min", metrics_mean_v3["business_cost_min"])
    mlflow.log_metric("optimal_threshold", metrics_mean_v3["optimal_threshold"])
    
    # Sauvegarder le modèle ET l'enregistrer dans le Model Registry
    model_info = mlflow.sklearn.log_model(
        final_model_v3,
        artifact_path="logistic_regression_v3_robust_scaler",
        registered_model_name="regression"  # Enregistrement automatique dans Model Registry
    )
    
    print(f"\n✓ Modèle V3 enregistré dans MLflow Model Registry sous le nom 'regression'")
    print(f"  AUC (CV): {metrics_mean_v3['auc']:.4f}")
    print(f"  F1 (CV): {metrics_mean_v3['f1_score']:.4f}")
    print(f"  Recall (CV): {metrics_mean_v3['recall_class1']:.4f}")
    print(f"  Business Cost Min (CV): {metrics_mean_v3['business_cost_min']:.2f}")
    print(f"\n  Model URI: {model_info.model_uri}")
    print(f"  ℹ️  Ce modèle est maintenant disponible dans le Model Registry")
    print(f"     Accessible via: mlflow.sklearn.load_model('models:/regression/latest')")


✓ Modèle final LogisticRegression V3 entraîné sur l'ensemble train complet
  Nombre de features: 724
  Intercept: -0.000000
  Norme des coefficients: 0.000000


2026/02/06 02:11:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/06 02:11:49 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'regression' already exists. Creating a new version of this model...
2026/02/06 02:11:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: regression, version 3
Created version '3' of model 'regression'.



✓ Modèle V3 enregistré dans MLflow Model Registry sous le nom 'regression'
  AUC (CV): 0.5424
  F1 (CV): 0.1523
  Recall (CV): 0.7252
  Business Cost Min (CV): 1636.60

  Model URI: models:/m-6f0e559865f84c4a9bae981ffb44747e
  ℹ️  Ce modèle est maintenant disponible dans le Model Registry
     Accessible via: mlflow.sklearn.load_model('models:/regression/latest')
🏃 View run V3_LogisticRegression_RobustScaler_Final at: http://127.0.0.1:5000/#/experiments/1/runs/6d7ce4bf0fa94725a3b69b3f85e5bdc8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [ ]:
# ============================================================================
# TABLEAU COMPARATIF FINAL: Toutes les versions (V1, V2.1, V2.2, V3)
# ============================================================================

print("\n" + "="*130)
print("TABLEAU COMPARATIF FINAL: Toutes les versions")
print("="*130)

comparison_data_final = {
    "Version": ["V1 Baseline", "V2.1 Class Weight", "V2.2 SMOTE", "V3 RobustScaler + CW"],
    "Scaling": ["StandardScaler", "StandardScaler", "StandardScaler", "RobustScaler"],
    "Imbalance": ["None", "class_weight", "SMOTE", "class_weight"],
    "AUC": [
        f"{metrics_mean['auc']:.4f} ± {metrics_std['auc']:.4f}",
        f"{metrics_mean_v2_1['auc']:.4f} ± {metrics_std_v2_1['auc']:.4f}",
        f"{metrics_mean_v2_2['auc']:.4f} ± {metrics_std_v2_2['auc']:.4f}",
        f"{metrics_mean_v3['auc']:.4f} ± {metrics_std_v3['auc']:.4f}",
    ],
    "F1-Score": [
        f"{metrics_mean['f1_score']:.4f} ± {metrics_std['f1_score']:.4f}",
        f"{metrics_mean_v2_1['f1_score']:.4f} ± {metrics_std_v2_1['f1_score']:.4f}",
        f"{metrics_mean_v2_2['f1_score']:.4f} ± {metrics_std_v2_2['f1_score']:.4f}",
        f"{metrics_mean_v3['f1_score']:.4f} ± {metrics_std_v3['f1_score']:.4f}",
    ],
    "Recall": [
        f"{metrics_mean['recall_class1']:.4f} ± {metrics_std['recall_class1']:.4f}",
        f"{metrics_mean_v2_1['recall_class1']:.4f} ± {metrics_std_v2_1['recall_class1']:.4f}",
        f"{metrics_mean_v2_2['recall_class1']:.4f} ± {metrics_std_v2_2['recall_class1']:.4f}",
        f"{metrics_mean_v3['recall_class1']:.4f} ± {metrics_std_v3['recall_class1']:.4f}",
    ],
    "Coût Métier": [
        f"{metrics_mean['business_cost_min']:.2f} ± {metrics_std['business_cost_min']:.2f}",
        f"{metrics_mean_v2_1['business_cost_min']:.2f} ± {metrics_std_v2_1['business_cost_min']:.2f}",
        f"{metrics_mean_v2_2['business_cost_min']:.2f} ± {metrics_std_v2_2['business_cost_min']:.2f}",
        f"{metrics_mean_v3['business_cost_min']:.2f} ± {metrics_std_v3['business_cost_min']:.2f}",
    ]
}

comparison_df_final = pd.DataFrame(comparison_data_final)
print(comparison_df_final.to_string(index=False))
print("="*130)

# Déterminer la meilleure version finale
all_metrics = {
    "V1": metrics_mean,
    "V2.1": metrics_mean_v2_1,
    "V2.2": metrics_mean_v2_2,
    "V3": metrics_mean_v3
}

best_auc_v = max(all_metrics.items(), key=lambda x: x[1]['auc'])
best_f1_v = max(all_metrics.items(), key=lambda x: x[1]['f1_score'])
best_recall_v = max(all_metrics.items(), key=lambda x: x[1]['recall_class1'])
best_cost_v = min(all_metrics.items(), key=lambda x: x[1]['business_cost_min'])

print("\n✓ Meilleure version par métrique:")
print(f"  AUC:              {best_auc_v[0]} ({best_auc_v[1]['auc']:.4f})")
print(f"  F1-Score:         {best_f1_v[0]} ({best_f1_v[1]['f1_score']:.4f})")
print(f"  Recall Classe 1:  {best_recall_v[0]} ({best_recall_v[1]['recall_class1']:.4f})")
print(f"  Coût Métier Min:  {best_cost_v[0]} ({best_cost_v[1]['business_cost_min']:.2f})")

# Sélection dynamique basée sur le Coût Métier (métrique métier principale)
best_overall_version = best_cost_v[0]

version_names = {
    "V1": "V1 Baseline",
    "V2.1": "V2.1 StandardScaler + class_weight='balanced'",
    "V2.2": "V2.2 StandardScaler + SMOTE",
    "V3": "V3 RobustScaler + class_weight='balanced'"
}

print("\n" + "="*130)
print(f"✓ MODÈLE FINAL SÉLECTIONNÉ: {version_names[best_overall_version]}")
print(f"  Basé sur le Coût Métier (métrique métier principale): {best_cost_v[1]['business_cost_min']:.2f}")

if best_overall_version == "V3":
    print("  Enregistré dans MLflow Model Registry sous le nom: 'regression'")
else:
    print(f"  Note: V3 a été enregistré dans Model Registry pour démonstration,")
    print(f"        mais {version_names[best_overall_version]} a de meilleures performances")

print("="*130)


TABLEAU COMPARATIF FINAL: Toutes les versions
             Version        Scaling    Imbalance             AUC        F1-Score          Recall     Coût Métier
         V1 Baseline StandardScaler         None 0.7010 ± 0.0038 0.0631 ± 0.0355 0.0361 ± 0.0202 1511.00 ± 34.85
   V2.1 Class Weight StandardScaler class_weight 0.6957 ± 0.0117 0.2367 ± 0.0078 0.5832 ± 0.0305 1164.40 ± 32.60
          V2.2 SMOTE StandardScaler        SMOTE 0.6890 ± 0.0104 0.2386 ± 0.0104 0.5600 ± 0.0297 1168.60 ± 36.73
V3 RobustScaler + CW   RobustScaler class_weight 0.5424 ± 0.0245 0.1523 ± 0.0051 0.7252 ± 0.0298 1636.60 ± 52.71

✓ Meilleure version par métrique:
  AUC:              V1 (0.7010)
  F1-Score:         V2.2 (0.2386)
  Recall Classe 1:  V3 (0.7252)
  Coût Métier Min:  V2.1 (1164.40)

✓ MODÈLE FINAL SÉLECTIONNÉ: V1 Baseline
  Basé sur l'AUC (métrique principale): 0.7010
  Note: V3 a été enregistré dans Model Registry pour démonstration,
        mais V1 Baseline a de meilleures performances
